In [12]:
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv(os.path.join("..", "data", "csv", "scikit", "compas_recidive_two_years_sanitize_age_category_jail_time_decile_score.csv"))

In [14]:
df_binary = df[(df["race"] == "Caucasian") | (df["race"] == "African-American")]

del df_binary['c_jail_in']
del df_binary['c_jail_out']

##separated class from the rests of the features
#remove unnecessary dimensions from Y -> only the decile_score remains
Y = df_binary['decile_score']
del df_binary['decile_score']
del df_binary['two_year_recid']
del df_binary['score_text']

S = df_binary['race']
del df_binary['race']

In [15]:
df_binary.head()

,sex,age_cat,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_time (days),date_dif_in_jail,c_charge_degree,is_recid
1,Male,25 - 45,0,0,0,0,-1,10.077384,10,F,1
2,Male,Less than 25,0,0,1,4,-1,1.085764,1,F,1
4,Male,25 - 45,0,0,0,14,-1,6.298681,6,F,1
6,Female,25 - 45,0,0,0,0,-1,2.953611,3,M,0
7,Male,25 - 45,0,0,0,0,-1,1.080451,1,F,0


In [16]:
encod = preprocessing.OrdinalEncoder()
encod.fit(df_binary)
X = encod.transform(df_binary)
X = pd.DataFrame(X)
X.columns = df_binary.columns
X.head()

,sex,age_cat,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_time (days),date_dif_in_jail,c_charge_degree,is_recid
0,1.0,0.0,0.0,0.0,0.0,0.0,29.0,4127.0,10.0,0.0,1.0
1,1.0,2.0,0.0,0.0,1.0,4.0,29.0,2204.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,14.0,29.0,3920.0,6.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,29.0,3503.0,3.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,29.0,2184.0,1.0,0.0,0.0


In [17]:
# Trained on X_train with random state 42, so we'll keep that
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(np.unique(Y_train))
print(np.unique(Y_test))

(3694, 11)
(1584, 11)
[ 1  2  3  4  5  6  7  8  9 10]
[ 1  2  3  4  5  6  7  8  9 10]


In [18]:
from secml.data import CDataset
tr_set_secML = CDataset(X_train,Y_train)
ts_set_secML = CDataset(X_test,Y_test)

# Create a surrogate classifier

In [19]:
# Creation of the multiclass classifier
from secml.ml.classifiers import CClassifierSVM
from secml.ml.classifiers.multiclass import CClassifierMulticlassOVA
from secml.ml.kernel import CKernelRBF
clf = CClassifierMulticlassOVA(CClassifierSVM, kernel=CKernelRBF())

the classifier is declared, now we will try to find the best set of parameters such that it performs well on a portion of the training set

In [34]:
# Parameters for the Cross-Validation procedure
xval_params = {'C': [1e-4, 1e-3, 1e-2, 0.1, 1], 'kernel.gamma': [0.01, 0.1, 1, 10, 100, 1e3]}

# Let's create a 3-Fold data splitter
random_state = 999

from secml.data.splitter import CDataSplitterKFold
xval_splitter = CDataSplitterKFold(num_folds=3, random_state=random_state)

# Select and set the best training parameters for the classifier
print("Estimating the best training parameters...")
best_params = clf.estimate_parameters(
    dataset=tr_set_secML,
    parameters=xval_params,
    splitter=xval_splitter,
    metric='accuracy',
    perf_evaluator='xval'
)
print("The best training parameters are: ", best_params)

Estimating the best training parameters...
The best training parameters are:  {'C': 0.001, 'kernel.gamma': 1}


In [35]:
print(clf.get_params())
print(clf.num_classifiers)

{'preprocess': None}
10


In [36]:
# Metric to use for training and performance evaluation
from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()


# Train the classifier
clf.fit(tr_set_secML)
print(clf.num_classifiers)

# Compute predictions on a test set
y_pred = clf.predict(ts_set_secML.X)

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts_set_secML.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

10
Accuracy on test set: 10.04%


# Prepare attack configuration

In [11]:
noise_type = 'l2'  # Type of perturbation 'l1' or 'l2'
dmax = 0.4  # Maximum perturbation
lb, ub = 0, 1  # Bounds of the attack space. Can be set to `None` for unbounded
y_target = None  # None if `error-generic` or a class label for `error-specific`

# Should be chosen depending on the optimization problem
solver_params = {
    'eta': 0.3,
    'eta_min': 0.1,
    'eta_max': None,
    'max_iter': 100,
    'eps': 1e-4
}

# Run attack

In [12]:
from secml.adv.attacks.evasion import CAttackEvasionPGDLS
pgd_ls_attack = CAttackEvasionPGDLS(
    classifier=clf,
    surrogate_classifier=clf,
    surrogate_data=tr_set_secML,
    distance=noise_type,
    dmax=dmax,
    lb=lb, ub=ub,
    solver_params=solver_params,
    y_target=y_target)

In [13]:
nb_attack=25
result_pts=np.empty(nb_attack)
result_class=np.empty(nb_attack)

#take a point at random being the starting point of the attack
import random
for nb_iter in range(0,nb_attack-1):
    rn = random.randint(0,ts_set_secML.num_samples)
    x0,y0 = ts_set_secML[rn,:].X, ts_set_secML[rn,:].Y
    
    y_pred_pgdls, _, adv_ds_pgdls, _ = pgd_ls_attack.run(x0, y0)
    results_pts[nb_iter] = adv_ds_pgdls
    result_class[nb_iter] = y_pred_pgdls

ValueError: x_init CArray([1.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 5.999529e+00
 2.899736e+01 4.249600e+03 1.399878e+01 0.000000e+00 1.000000e+00]) is outside of feasible domain.

In [ ]:
# Run the evasion attack on x0
y_pred_pgdls, _, adv_ds_pgdls, _ = pgd_ls_attack.run(x0, y0)

add a block with some constraints if needed to keep dimensions in predefined ranges of values; force the value of some dimensions if constraints are specified; etc.

In [ ]:
print("Original x0 label: ", y0.item())
print("Adversarial example label (PGD-LS): ", y_pred_pgdls.item())

print("Number of classifier gradient evaluations: {:}"
      "".format(pgd_ls_attack.grad_eval))